In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'

from dataloader import *
from models import load_model
from metrics import *
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import scipy
import class_names
from utils import *

import json
import urllib
from preprocessing import preprocess_images_any_dataset

/home/davidva/miniconda3/envs/anaconda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
experiment_list = os.listdir('/home/davidva/experiments_David')
experiment_list.remove('.ipynb_checkpoints')
header_exists_flag = 0  # indicates whether the combines CSV file has a header of "Dataset, Model, Accuracy..."
# print(experiment_list)
with open('/home/davidva/vscode_projects/reliableML/all_experiments.csv', 'w') as f:
    wf = csv.writer(f, delimiter = ',')
    for folder in experiment_list[4:]:
        results_path = '/home/davidva/experiments_David/' + folder + '/results'
        file_list = os.listdir(results_path)
        end_of_data_name_index = folder.index('_')
        with open(results_path + '/' + file_list[2], mode="r") as csv_file:
            reader = csv.reader(csv_file) #this is the reader object
            line_count = 0
            for row_num, row in enumerate(reader):
                if header_exists_flag == 0 and row_num == 0:
                    header_exists_flag = 1
                    row = ['Dataset', 'Model'] + row
                    wf.writerow(row)
                if row_num == 1:
                    sub_strings = folder.split('_')
                    model_name = sub_strings[2] + '_resnet' if 'wide' in sub_strings[2] else sub_strings[2]
                    data_name = sub_strings[1]
                    row = [data_name, model_name] + row
                    wf.writerow(row)

In [37]:
f = open('/home/davidva/datasets/imagenet_vid_ytbb_robust/imagenet-vid-robust/misc/imagenet_vid_class_index.json')
vid_class_index = json.load(f)
g = open('/home/davidva/datasets/imagenet_vid_ytbb_robust/imagenet-vid-robust/misc/rev_wnid_map.json')
class_name_converter = json.load(g)
ground_truth = str(1)
input_to_class_name_converter = vid_class_index[ground_truth][0]
print(input_to_class_name_converter)
class_name_converter[input_to_class_name_converter]

n02419796


['n02422106', 'n02422699', 'n02423022']

In [34]:
import class_names


h = open('/home/davidva/datasets/imagenet_vid_ytbb_robust/imagenet-vid-robust/misc/wnid_map.json')
wnid_map = json.load(h)

lll = []
for class_name in class_names.all_wnids:
    lll.append(True if class_name in wnid_map.keys() else False)

288

In [12]:
def main():
    # Define device, batch size and directory path for ImageNet validation set
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    
    base_exp_dir = '/home/davidva/experiments_David'

    # Define the batch size
    batch_size = 128

    # Load imagenet labels as real classes
    imagenet_labels = load_imagenet_labels()  # 'tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead shark'...

    data_names = ['imagenetv2-matched-frequency-format-val', 'imagenetv2-threshold0.7-format-val', 'imagenetv2-top-images-format-val', 'imagenetsketch/sketch', 'imagenet-r', 'imagenet-a', 'imagenet']
    data_names = ['imagenetsketch']

    for data_name in data_names:
        if data_name in ['imagenet', 'imagenetv2-matched-frequency-format-val', 'imagenetv2-threshold0.7-format-val', 'imagenetv2-top-images-format-val', 'imagenetsketch']:
            ds_specific_mask = [True] * 1000
            ds_specific_labels = imagenet_labels  # the classes of the specific dataset that we're working with
        elif data_name == 'imagenet-r':
            ds_specific_r_mask = class_names.imagenet_r_mask
            ds_specific_labels = class_names.imagenet_r_labels
        elif data_name == 'imagenet-a':  # note: if the hidden file .ipynb_checkpoints is inside of the folder of the imagenet-a data,
                                         # it should be removed from there in order to create the DataFolder object
            ds_specific_mask = class_names.imagenet_a_mask
            ds_specific_labels = class_names.imagenet_a_labels
        elif data_name == 'imagenet_vid_robust':
            ds_specific_mask = class_names.imagenet_vid_robust_mask
            ds_specific_labels = class_names.imagenet_vid_robust_labels
            f = open('/home/davidva/datasets/imagenet_vid_ytbb_robust/imagenet-vid-robust/misc/imagenet_vid_class_index.json')
            vid_class_index_to_nid = json.load(f)  # converts class index (between 0 to 29) to imagenet-vid-robust nid (e.g. n029...)
            g = open('/home/davidva/datasets/imagenet_vid_ytbb_robust/imagenet-vid-robust/misc/rev_wnid_map.json')
            class_name_converter = json.load(g)  # converts imagenet-vid-robust nid to a list of imagenet nids
        
        if data_name == 'imagenet':
            val_dir = '/datasets/ImageNet/val/'
        elif data_name == 'imagenetsketch':
            val_dir = '/home/davidva/datasets/imagenetsketch/sketch'
        else:
            val_dir = '/home/davidva/datasets/' + data_name
        print('data_name is: ' + data_name + ' and val_dir is: ' + val_dir)

        model_names = ['mobilenet_v2', 'resnet50', 'resnet34']
    
        for model_name in model_names:
            # Experiment name
            curr_time = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
            exp_base_name = 'predict_' + data_name + '_' + model_name
            exp_name = exp_base_name + '_' + curr_time
            exp_dir = os.path.join(base_exp_dir, exp_name)
            results_dir = os.path.join(exp_dir, 'results')
            embed_dir = os.path.join(results_dir, 'val_embeddings')
        
            # Create experiment directory
            os.makedirs(exp_dir, exist_ok=True)
            os.makedirs(results_dir, exist_ok=True)
            os.makedirs(embed_dir, exist_ok=True)
                
            # Load the ImageNet validation dataset with defined transformations
            val_dataset = ImageFolder(val_dir, transform=preprocess_images_any_dataset(model_name))
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=5)
    
            # Load model
            model = load_model(model_name).to(device)
            model.eval()
            
            # Register the hook at the penultimate layer of the model
            if model_name in ['mobilenet_v2']:
                layer = model.classifier[0]
            elif model_name in ['densenet121']:
                layer = model.features   # model.features.norm5 sucks, features.denseblock3.denselayer24 is okay, features gives 1000-of-shape output, model.features.denseblock3.denselayer24.conv2 is okay
            elif model_name in ['vgg16', 'vgg19']:
                layer = model.classifier[-2]
            else:
                layer = model.avgpool
            
            # Prepare DataFrame to store results
            results = []
            total_correct = 0
            batch_num = 0
            all_entropies = np.empty([0])
            all_means = np.empty([0])
            all_variances = np.empty([0])
            all_skewnesses = np.empty([0])
            all_kurtosises = np.empty([0])
            all_top5_entropies = np.empty([0])
            all_top5_means = np.empty([0])
            all_top5_variances = np.empty([0])
            all_top5_skewnesses = np.empty([0])
            all_top5_kurtosises = np.empty([0])
    
            # Disable gradients for faster inference
            with torch.no_grad():
                for batch_idx, (images, labels) in enumerate(tqdm(val_loader, desc=f"Processing {exp_name}")):
                    images = images.to(device)
                    labels = labels.to(device)
    
                    handle = layer.register_forward_hook(get_embedding_hook(batch_idx, batch_size, embed_dir))
    
                    # Run forward pass through the model and calculate probabilities
                    output = model(images)
                    probabilities = torch.nn.functional.softmax(output, dim=1)
                    probabilities = probabilities * torch.tensor(ds_specific_mask).to(device)  # using only the classes that are in both datasets
        
                    # Get top-5 predictions and confidences and some statistics
                    top5_prob, top5_classes = torch.topk(probabilities, 5)
                    _, _, top5_means, top5_variances, top5_skewnesses, top5_kurtosises = scipy.stats.describe(top5_prob.detach().cpu().numpy() , axis=1)
                    all_top5_means = np.concatenate((all_top5_means, top5_means))
                    all_top5_variances = np.concatenate((all_top5_variances, top5_variances))
                    all_top5_skewnesses = np.concatenate((all_top5_skewnesses, top5_skewnesses))
                    all_top5_kurtosises = np.concatenate((all_top5_kurtosises, top5_kurtosises))
                    top5_entropies = scipy.stats.entropy(top5_prob.detach().cpu().numpy(), axis=1)
                    all_top5_entropies = np.concatenate((all_top5_entropies, top5_entropies))
                    
                    _, _, means, variances, skewnesses, kurtosises = scipy.stats.describe(probabilities.detach().cpu().numpy() , axis=1)
                    all_means = np.concatenate((all_means, means))
                    all_variances = np.concatenate((all_variances, variances))
                    all_skewnesses = np.concatenate((all_skewnesses, skewnesses))
                    all_kurtosises = np.concatenate((all_kurtosises, kurtosises))
                    entropies = scipy.stats.entropy(probabilities.detach().cpu().numpy(), axis=1)
                    all_entropies = np.concatenate((all_entropies, entropies))
        
                    # Iterate over each image in the batch
                    for i in range(len(images)):
                        image_idx = batch_idx * batch_size + i
                        image_path = val_dataset.imgs[i + batch_num * batch_size][0]
                        # true_class = val_dataset.classes[labels[i].item()]
                        true_class = ds_specific_labels[labels[i].item()]  # e.g. labels[i].item()=0. true_class=goldfish
                        predictions = {"image_path": image_path,
                                       "true_class": true_class,
                                       "embeddings_path":  os.path.join(embed_dir, f"{image_idx}_embeddings.npy")}
                        
                        for j in range(5):
                            # predictions[f"top{j+1}_prediction_class"] = val_dataset.classes[top5_classes[i, j].item()]
                            prediction_class = imagenet_labels[top5_classes[i, j].item()]
                            predictions[f"top{j+1}_prediction_class"] = prediction_class
                            predictions[f"top{j+1}_confidence"] = top5_prob[i, j].item()

                            # Check if top-1 prediction is correct
                            if data_name == 'imagenet_vid_robust':
                                if j == 0:
                                    vid_robust_class_nid = vid_class_index_to_nid[str(labels[i].item())][0]  # e.g. 'n02419796'
                                    ground_truth_classes = class_name_converter[vid_robust_class_nid] # e.g. ['n02422106', 'n02422699', 'n02423022']
                                    if prediction_class in ground_truth_classes:
                                        total_correct += 1
                            elif j == 0 and prediction_class == true_class:
                                total_correct += 1
                        results.append(predictions)
                    batch_num += 1
                    handle.remove()  # Unregister the hook after processing the batch
                    break
                                
            
            # Convert results to DataFrame and save to CSV
            results_df = pd.DataFrame(results)
            results_df.to_csv(os.path.join(results_dir, f"full_val_predictions_{exp_base_name}.csv"), index=False)
        
            # Calculate average and std of the top1 confidences across the val set2
            avg_confidence = results_df["top1_confidence"].mean()
            std_confidence = results_df["top1_confidence"].std()
            accuracy = total_correct / len(val_dataset)
            average_top5_mean = np.mean(all_top5_means)
            average_top5_variance = np.mean(all_top5_variances)
            average_top5_skewness = np.mean(all_top5_skewnesses)
            average_top5_kurtosis = np.mean(all_top5_kurtosises)
            average_top5_entropy = np.mean(all_top5_entropies)
            average_mean = np.mean(all_means)
            average_variance = np.mean(all_variances)
            average_skewness = np.mean(all_skewnesses)
            average_kurtosis = np.mean(all_kurtosises)
            average_entropy = np.mean(all_entropies)
            top2_grad_confidence = (results_df["top1_confidence"] - results_df["top2_confidence"]).mean()
    
            # Save average and std confidences in a new DataFrame and some more statistics
            confidence_df = pd.DataFrame({
                "Accuracy": [accuracy],
                "Average Confidence": [avg_confidence],
                "Standard Deviation Confidence": [std_confidence],
                "Average Entropy": [average_entropy],
                "Average mean": [average_mean],
                "Average variance": [average_variance],
                "Average skewness": [average_skewness],
                "Average kurtosis": [average_kurtosis],
                "Average top5 Entropy": [average_top5_entropy],
                "Average top5 mean": [average_top5_mean],
                "Average top5 variance": [average_top5_variance],
                "Average top5 skewness": [average_top5_skewness],
                "Average top5 kurtosis": [average_top5_kurtosis],
                "top2_grad_confidence": [top2_grad_confidence]
            })
            confidence_df.to_csv(os.path.join(results_dir, f"val_confidence_summary_{exp_base_name}.csv"), index=False)
    
            # Handle embeddings
            embeddings = np.array([np.load(file) for file in results_df['embeddings_path']])
            # assuming embeddings is a 4D numpy array
            embeddings = embeddings.reshape(embeddings.shape[0], -1)
            print(embeddings)
            print(embeddings.shape)
            continue
            average, covariance = calculate_activation_statistics(embeddings)
            np.save(os.path.join(results_dir, 'embeddings_covariance.npy'), covariance)
            np.save(os.path.join(results_dir, 'embeddings_average.npy'), average)
            # np.save(os.path.join(results_dir, 'tot_embeddings_2D_np_array.npy'), embeddings)
            
            # Create a new DataFrame for embeddings
            # df_embeddings = pd.DataFrame(*******, columns=np.arange(0, embeddings.shape[1]))
            # df_embeddings['true_class'] = results_df['true_class']
            # df_embeddings['predicted_class'] = results_df['top1_prediction_class']
            # df_embeddings['image_path'] = results_df['image_path']
            # # Save as CSV
            # df_embeddings.to_csv(os.path.join(results_dir, f"val_embeddings_2d_{exp_base_name}.csv"), index=False)

In [13]:
if __name__ == '__main__':
    main()

data_name is: imagenetsketch and val_dir is: /home/davidva/datasets/imagenetsketch/sketch


/home/davidva/miniconda3/envs/anaconda/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Processing predict_imagenetsketch_mobilenet_v2_22-07-2023_18-57-59:   0%|   | 0/398 [00:16<?, ?it/s]


[[0.02671015 0.00171657 0.0460147  ... 0.         0.         0.        ]
 [0.         0.02037799 0.0677322  ... 0.         0.         0.        ]
 [0.41246688 0.0139725  0.39630803 ... 0.         0.35750774 0.01537326]
 ...
 [0.07032369 0.         0.6472685  ... 0.00488729 0.21846849 0.01720898]
 [0.         0.04267349 0.         ... 0.06296569 0.10565615 0.        ]
 [0.         0.         0.         ... 0.24524435 0.08623537 0.        ]]
(128, 1280)


/home/davidva/miniconda3/envs/anaconda/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Processing predict_imagenetsketch_resnet50_22-07-2023_18-58-18:   0%|       | 0/398 [00:16<?, ?it/s]


[[0.07946252 0.00908436 0.11294913 ... 0.03491938 0.         0.3887446 ]
 [0.9069552  0.09604716 0.7782015  ... 0.07150039 0.         0.5465018 ]
 [1.0958493  0.09429942 0.06916237 ... 0.20279767 0.         0.5502091 ]
 ...
 [0.03972644 0.15649162 0.09300479 ... 0.01561494 0.08422109 0.1410367 ]
 [0.02537842 0.         0.11367818 ... 0.         0.         0.42014655]
 [0.01596968 0.14475392 0.07495753 ... 0.0291111  0.         0.        ]]
(128, 2048)


/home/davidva/miniconda3/envs/anaconda/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Processing predict_imagenetsketch_resnet34_22-07-2023_18-58-35:   0%|       | 0/398 [00:18<?, ?it/s]

[[0.653874   0.31201592 0.16038497 ... 1.0917758  0.3057381  0.21315517]
 [1.4613996  0.12096184 0.78954923 ... 6.0389     1.9267793  0.8895854 ]
 [1.2781454  2.9755695  0.6889137  ... 2.5450897  0.72125584 1.2295638 ]
 ...
 [0.07453956 0.18535359 0.35546103 ... 4.2711263  0.25953907 1.3707743 ]
 [0.33146438 1.0275772  0.5090131  ... 0.5566723  0.19399302 2.6344025 ]
 [0.3677118  1.3870504  0.5162372  ... 0.9433905  0.18702917 2.9107087 ]]
(128, 512)


In [33]:
# for layer in k.children():
#     print(layer)

model_names = ['resnet50', 'resnet18', 'resnet34', 'resnet101', 'resnet152', 'vgg16', 'vgg19', 'alexnet', 'resnext', 'wide_resnet', 'densenet121', 'googlenet', 'mobilenet_v2']

for model_name in model_names:
    k = load_model(model_name)
    layers_list = []
    for name, _ in k.named_children():
        if not name.startswith('params'):
            layers_list.append(name)
    
    print(model_name, layers_list[-2], layers_list[-1])


resnet50 avgpool fc
resnet18 avgpool fc
resnet34 avgpool fc
resnet101 avgpool fc
resnet152 avgpool fc
vgg16 avgpool classifier
vgg19 avgpool classifier
alexnet avgpool classifier
resnext avgpool fc
wide_resnet avgpool fc
densenet121 features classifier
googlenet dropout fc
mobilenet_v2 features classifier
